In [34]:
from smolagents import CodeAgent, LiteLLMModel, HfApiModel
from tools.atom_feed_tool import AtomFeedTool
from tools.visit_web_page_tool import VisitWebPageTool
from tools.visit_web_page_sync_tool import VisitWebPageSyncTool
from tools.retriever_tool import RetrieverTool
from smolagents import FinalAnswerTool
import yaml
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import os
from threading import Thread

In [35]:
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
# model = HfApiModel(
#         max_tokens=1000,
#         temperature=0,
#         model_id='Qwen/Qwen2.5-Coder-32B-Instruct',
#     )

In [37]:
model = LiteLLMModel(
        model_id="ollama/qwen2.5-coder",  # Or try other Ollama-supported models
        api_base="http://localhost:11434",  # Default Ollama local server
        num_ctx=8192,
)

In [38]:
with open("prompt-cloudcast.yaml", "r") as f:
    prompt_templates = yaml.safe_load(f)

In [39]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load the Chroma vector store from disk
vector_store = Chroma(
    collection_name="langchain",  # Replace with your collection name
    embedding_function=embeddings,
    persist_directory="./chroma_db"
)

In [40]:
agent = CodeAgent(
    model=model,
    tools=[
        AtomFeedTool(),
        VisitWebPageSyncTool(),
        FinalAnswerTool(),
        RetrieverTool(vector_store=vector_store)
    ],
    prompt_templates=prompt_templates,
    additional_authorized_imports=["feedparser", "requests", "json", "re", "html", "playwright", "asyncio"],
    add_base_tools=False,
)

In [41]:
agent_run = agent.run("visit https://www.thecloudcast.net and find the podcast on security", stream=True)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ visit https://www.thecloudcast.net and find the podcast on security                                             │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5-coder ───────────────────────────────────────────────────────────────────────────╯

In [42]:
next(agent_run)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━